In [1]:
import os, sys
import gc
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
sys.path.append('../')
import torch

In [3]:
from get_algos import get_all_algos , run_experiment ,call_paths, plot_mean

In [4]:
import gym
from sb3_contrib import QRDQN ,RecurrentPPO, TRPO
from stable_baselines3.common.vec_env import VecFrameStack,VecVideoRecorder
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.monitor import Monitor

In [5]:
env_name='Breakout-v0'

### 1. Test
<ol>
    <li>Create the environment</li>
    <li>For each algorithm:</li>
    <ol><li>Load the model</li>
        <li>Evaluate the model for 5 sample iterations </li>
        <li>Output the score  for each algo and each iteration</li>
        <li>Render the video and store it </li>
    </ol>
</ol> 

In [6]:
env_recurrent_type={'Discrete':['MaskablePPO', 'QRDQN', 'RecurrentPPO', 'TRPO'],'Box':['ARS', 'RecurrentPPO', 'TQC', 'TRPO'], 
                    'MultiDiscrete':['MaskablePPO', 'RecurrentPPO', 'TRPO'],'MultiBinary':['MaskablePPO', 'RecurrentPPO', 'TRPO']}
def get_algo_recurrent(env):
    act_space=str(type(env.action_space)).split('.')[3].split("'")[0]
    return env_recurrent_type[act_space]

algo_list=get_algo_recurrent(gym.make(env_name))
algo_list.remove('MaskablePPO') if 'MaskablePPO' in algo_list else algo_list
print(algo_list)

['QRDQN', 'RecurrentPPO', 'TRPO']


### 3. Testing

In [7]:
def testing_model(algo_list, env_name,n_steps,vid_length):
    total_rewards={}
    for algo_name in reversed(algo_list):
        algo_rewards=[]
        if algo_name =='QRDQN' or 'TRPO':
            device='cpu'
        else:
            device='cuda' #GPU was unable to load the entire experience relay so moved to virtual memory on disk
        policy='CnnLstmPolicy' if algo_name=='RecurrentPPO' else 'CnnPolicy'
        log_path, render_path, model_path=call_paths(algo_name,env_name,n_steps)
        env=VecVideoRecorder(VecFrameStack(make_atari_env(env_name,monitor_dir=render_path),n_stack=6),render_path,record_video_trigger=lambda step: step == 0,video_length=vid_length)
        model_name=env_name+"_"+algo_name+"_model"
        model=eval(algo_name).load(os.path.join(model_path, model_name),env,device=device)
        for i in range(5):
            state=env.reset()
            epi_rewards=0
            while True:
                action,_=model.predict(state)
                state,reward,done,_=env.step(action)
                epi_rewards+=reward[0]
                env.render()
                if done:
                    algo_rewards.append(epi_rewards)
                    break
        total_rewards[algo_name]=algo_rewards
        del model
        env.close()
    return total_rewards

In [8]:
test=total_rewards=testing_model(algo_list, env_name,n_steps=200000,vid_length=1000)

Wrapping the env in a VecTransposeImage.


E:\Anaconda\envs\ai_gym\lib\site-packages\gym\envs\atari\environment.py:269: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "


Wrapping the env in a VecTransposeImage.
Wrapping the env in a VecTransposeImage.


E:\Anaconda\envs\ai_gym\lib\site-packages\stable_baselines3\common\buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 84.69GB > 10.47GB
  "This system does not have apparently enough memory to store the complete "


In [9]:
test

{'TRPO': [0.0, 0.0, 0.0, 0.0, 0.0],
 'RecurrentPPO': [0.0, 5.0, 0.0, 0.0, 0.0],
 'QRDQN': [0.0, 0.0, 2.0, 1.0, 0.0]}

We see that RecurrentPPO seems to consistently provide rewards with the highest average reward. Hence, we will use RecurrentPPO as the algorithm for subsequent training of <b> 5,000,000 </b>steps